## Proj. Asn. #2 : Colin Kirby

In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Window for Display.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Project Assignment 02: Colin Kirby")

    ctx = moderngl.create_context()
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)

    return ctx

ctx = init_window()

width, height = 800, 600
aspect_ratio = width / height


In [3]:
# Import & Load 3D Models.
vertex_data_platform, _ = getObjectData("cube.obj", normal=True, texture=True)
vertex_data_teapot, _ = getObjectData("teapot_with_texCoords.obj", normal=True, texture=True)
vertex_data_light, _ = getObjectData("20_icosahedron.obj")

Normal exists
texture exists
Normal exists
texture exists


In [4]:
# Initialize VAOs & Shader Programs.
vao_platform = None
vao_teapot = None
vao_light_source = None
program_platform = None
program_teapot = None
program_light_source = None

In [5]:
# Start w/ Point Light.
is_point_light = True

In [6]:
# Calculate 3D Position of Light Source.
def get_light_position(angle):
    distance = 12.0
    direction = glm.normalize(glm.vec3(1.0, 1.0, 0.0))
    rotation_axis = glm.vec3(0.0, 1.0, 0.0)
    rotation = glm.rotate(glm.mat4(1.0), angle, rotation_axis)
    position = rotation * glm.vec4(direction * distance, 1.0)
    return glm.vec3(position)

In [7]:
# Return Normalized Light Vector.
def get_light_direction(light_position):
    return glm.normalize(-light_position)

In [8]:
# Create Scaling Matrix.
light_scale_matrix = glm.scale(glm.mat4(1.0), glm.vec3(0.1))

In [9]:
# Light Source Shader Program.
program_light_source = ctx.program(
    vertex_shader="""
    #version 330
    in vec3 in_position;
    uniform mat4 model;
    uniform mat4 view;
    uniform mat4 proj;

    void main() {
        gl_Position = proj * view * model * vec4(in_position, 1.0);
    }
    """,
    fragment_shader="""
    #version 330
    out vec4 frag_color;
    void main() {
        frag_color = vec4(1.0);  // White color for the light source object
    }
    """
)

vertex_data_light_np = np.array(vertex_data_light, dtype='f4')
vbo_light_source = ctx.buffer(vertex_data_light_np.tobytes())
vao_light_source = ctx.vertex_array(
    program_light_source,
    [(vbo_light_source, '3f', 'in_position')],
)

In [10]:
# Set Angle Of Light to 0.
light_angle = 0.0

In [11]:
# Load Image Func.
def load_image(image_file, channels="RGBA", flip_x=False, flip_y=False):
    texture_img = pygame.image.load(image_file)
    if flip_x or flip_y:
        texture_img = pygame.transform.flip(texture_img, flip_x, flip_y)
    texture_data = pygame.image.tobytes(texture_img, channels, True)
    return texture_data, texture_img.get_size()

In [12]:
# Load Grass & Gold Images for Textures.
grass_texture_data, grass_texture_size = load_image("grass.jpg", channels="RGB", flip_y=True)
grass_texture = ctx.texture(grass_texture_size, 3, grass_texture_data)
grass_texture.build_mipmaps()

gold_texture_data, gold_texture_size = load_image("gold.jpg", channels="RGB", flip_y=True)
gold_texture = ctx.texture(gold_texture_size, 3, gold_texture_data)
gold_texture.build_mipmaps()

In [13]:
# Camera & Projection Matrices.
fov = 60.0
near_plane = 1.0
far_plane = 45.0

In [14]:
# Set Proj & View Matrix.
proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)
view_matrix = glm.lookAt(
    glm.vec3(0.0, 10.0, 20.0),  # Camera Position
    glm.vec3(0.0, 0.0, 0.0),    # Look at point
    glm.vec3(0.0, 1.0, 0.0)     # Up vector
)

In [15]:
# Vertex Data for Platform.
def set_vert_data_platform():
    global vao_platform, program_platform
    vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4')
    vbo_platform = ctx.buffer(vertex_data_platform_np.tobytes())

    program_platform = ctx.program(
        vertex_shader="""
        #version 330
        layout(location = 0) in vec3 in_position;
        layout(location = 1) in vec3 in_normal;
        layout(location = 2) in vec2 in_uv;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat4 light_view;
        uniform mat4 light_proj;

        out vec3 frag_normal;
        out vec3 frag_position;
        out vec2 frag_uv;
        out vec4 frag_pos_light_space;

        void main() {
            mat3 normal_matrix = transpose(inverse(mat3(model)));
            frag_normal = normalize(normal_matrix * in_normal);
            frag_position = vec3(model * vec4(in_position, 1.0));
            frag_uv = in_uv;
            frag_pos_light_space = light_proj * light_view * model * vec4(in_position, 1.0);
            gl_Position = proj * view * vec4(frag_position, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        in vec3 frag_position;
        in vec2 frag_uv;
        in vec4 frag_pos_light_space;

        out vec4 frag_color;

        uniform vec3 light_position;
        uniform vec3 light_direction;
        uniform bool is_point_light;
        uniform sampler2D material_texture;
        uniform sampler2D shadow_map;

        void main() {
            vec3 material_color = texture(material_texture, frag_uv).rgb;
            vec3 normal = normalize(frag_normal);
            vec3 light_dir;
            if (is_point_light) {
                light_dir = normalize(light_position - frag_position);
            } else {
                light_dir = normalize(-light_direction);
            }

            // Shadow calculation
            vec3 proj_coords = frag_pos_light_space.xyz / frag_pos_light_space.w;
            proj_coords = proj_coords * 0.5 + 0.5;  // Transform to [0,1]

            if (proj_coords.z > 1.0) {
                discard;
            }

            float closest_depth = texture(shadow_map, proj_coords.xy).r;
            float current_depth = proj_coords.z;
            float bias = 0.0001; // Bias to prevent shadow acne
            float shadow = current_depth - bias > closest_depth ? 0.0 : 1.0;

            // Compute lighting
            vec3 ambient = 0.1 * material_color;
            vec3 result;
            float diff = max(dot(normal, light_dir), 0.0);
            vec3 diffuse = diff * material_color;

            // Add ambient lighting regardless of shadow
            result = ambient + diffuse * shadow;

            frag_color = vec4(result, 1.0);
        }
        """
    )

    vao_platform = ctx.vertex_array(
        program_platform,
        [(vbo_platform, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')]
    )

In [16]:
# Vertex Data for Teapot.
def set_vert_data_teapot():
    global vao_teapot, program_teapot, vbo_teapot
    vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4')
    vbo_teapot = ctx.buffer(vertex_data_teapot_np.tobytes())

    program_teapot = ctx.program(
        vertex_shader="""
        #version 330
        layout(location = 0) in vec3 in_position;
        layout(location = 1) in vec3 in_normal;
        layout(location = 2) in vec2 in_uv;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat4 light_view;
        uniform mat4 light_proj;

        out vec3 frag_normal;
        out vec3 frag_position;
        out vec2 frag_uv;
        out vec4 frag_pos_light_space;

        void main() {
            mat3 normal_matrix = transpose(inverse(mat3(model)));
            frag_normal = normalize(normal_matrix * in_normal);
            frag_position = vec3(model * vec4(in_position, 1.0));
            frag_uv = in_uv;
            frag_pos_light_space = light_proj * light_view * model * vec4(in_position, 1.0);
            gl_Position = proj * view * vec4(frag_position, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        in vec3 frag_position;
        in vec2 frag_uv;
        in vec4 frag_pos_light_space;

        out vec4 color;

        uniform vec3 light_position;
        uniform vec3 light_direction;
        uniform bool is_point_light;
        uniform sampler2D material_texture;
        uniform sampler2D shadow_map;
        uniform bool metal;
        uniform vec3 eyePosition;

        const float shininess = 25.0;

        void main() {
            vec3 material_color = texture(material_texture, frag_uv).rgb;
            vec3 normal = normalize(frag_normal);
            vec3 light_dir;
            if (is_point_light) {
                light_dir = normalize(light_position - frag_position);
            } else {
                light_dir = normalize(-light_direction);
            }

            // Shadow calculation
            vec3 proj_coords = frag_pos_light_space.xyz / frag_pos_light_space.w;
            proj_coords = proj_coords * 0.5 + 0.5;  // Transform to [0,1]

            if (proj_coords.z > 1.0) {
                discard;
            }

            float closest_depth = texture(shadow_map, proj_coords.xy).r;
            float current_depth = proj_coords.z;
            float bias = 0.0001; // Bias to prevent shadow acne
            float shadow = current_depth - bias > closest_depth ? 0.0 : 1.0;

            // Compute lighting
            vec3 ambient = 0.1 * material_color;
            vec3 result;
            if (metal) {
                vec3 view_dir = normalize(eyePosition - frag_position);
                vec3 reflect_dir = reflect(-light_dir, normal);
                float spec = pow(max(dot(view_dir, reflect_dir), 0.0), shininess);
                vec3 specular = spec * material_color;

                // Add ambient lighting regardless of shadow
                result = ambient + specular * shadow;
            } else {
                float diff = max(dot(normal, light_dir), 0.0);
                vec3 diffuse = diff * material_color;

                // Add ambient lighting regardless of shadow
                result = ambient + diffuse * shadow;
            }

            color = vec4(result, 1.0);
        }
        """
    )

    vao_teapot = ctx.vertex_array(
        program_teapot,
        [(vbo_teapot, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')]
    )

In [17]:
# Sets the Vertex Data by Referencing Funcs.
set_vert_data_platform()
set_vert_data_teapot()

In [18]:
# Create Depth Texture & framebuffer for Shadow Mapping.
depth_texture_size = (2048, 2048)

depth_texture = ctx.depth_texture(depth_texture_size)
depth_texture.repeat_x = False
depth_texture.repeat_y = False
depth_texture.filter = (moderngl.LINEAR, moderngl.LINEAR)
depth_fbo = ctx.framebuffer(depth_attachment=depth_texture)

In [19]:
# Create Sampler Object for Dept Texture.
shadow_map_sampler = ctx.sampler(
    texture=depth_texture,
    filter=(moderngl.LINEAR, moderngl.LINEAR),
    compare_func='less_equal',
)

In [20]:
# Shader Program for Rendering lights POV.
shadow_map_program = ctx.program(
    vertex_shader="""
    #version 330
    in vec3 in_position;
    uniform mat4 model;
    uniform mat4 view;
    uniform mat4 proj;

    void main() {
        gl_Position = proj * view * model * vec4(in_position, 1.0);
    }
    """,
    fragment_shader="""
    #version 330
    void main() {
        // Empty main function
    }
    """
)

In [21]:
# Prepare VAOs for Shadow Mapping.
vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4').reshape(-1, 8)
positions_platform_np = vertex_data_platform_np[:, 0:3].flatten().astype('f4')
vbo_shadow_platform = ctx.buffer(positions_platform_np.tobytes())
vao_shadow_platform = ctx.vertex_array(
    shadow_map_program,
    [
        (vbo_shadow_platform, '3f', 'in_position'),
    ]
)

vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4').reshape(-1, 8)
positions_teapot_np = vertex_data_teapot_np[:, 0:3].flatten().astype('f4')
vbo_shadow_teapot = ctx.buffer(positions_teapot_np.tobytes())
vao_shadow_teapot = ctx.vertex_array(
    shadow_map_program,
    [
        (vbo_shadow_teapot, '3f', 'in_position'),
    ]
)

In [22]:
# Create Quad for Displaying Depth Texture.
quad_vertex_data = np.array([
    # positions   # tex_coords
    -1.0, -1.0,    0.0, 0.0,
     1.0, -1.0,    1.0, 0.0,
    -1.0,  1.0,    0.0, 1.0,
     1.0,  1.0,    1.0, 1.0,
], dtype='f4')

vbo_quad = ctx.buffer(quad_vertex_data.tobytes())

quad_program = ctx.program(
    vertex_shader="""
    #version 330
    in vec2 in_position;
    in vec2 in_texcoord;
    out vec2 v_texcoord;
    void main() {
        gl_Position = vec4(in_position, 0.0, 1.0);
        v_texcoord = in_texcoord;
    }
    """,
    fragment_shader="""
    #version 330
    in vec2 v_texcoord;
    out vec4 frag_color;
    uniform sampler2D depth_texture;
    void main() {
        float depth = texture(depth_texture, v_texcoord).r;
        frag_color = vec4(vec3(depth), 1.0);
    }
    """
)

vao_quad = ctx.vertex_array(
    quad_program,
    [
        (vbo_quad, '2f 2f', 'in_position', 'in_texcoord')
    ]
)


In [ ]:
# Main Rendering Loop.
running = True
while running:
    # Event Handler.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_l:
                is_point_light = not is_point_light

    # Update Light Position.
    light_angle += 0.0002
    light_position = get_light_position(light_angle)
    light_direction = get_light_direction(light_position)

    # Transformation Matrix for Platform.
    model_matrix_platform = glm.scale(glm.mat4(1.0), glm.vec3(10.0, 0.05, 10.0))
    model_matrix_platform = glm.translate(model_matrix_platform, glm.vec3(0.0, 0.1, 0.0))

    # Transformation Matrix for Teapot.
    model_matrix_teapot = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 0.0))
    model_matrix_teapot = glm.scale(model_matrix_teapot, glm.vec3(0.29))
    
    # Calculate Eye Position.
    eye_position = glm.vec3(glm.inverse(view_matrix)[3])

    # Pass 1: Render Scene from Light's Point of View to create Depth Texture Map.
    depth_fbo.use()
    ctx.viewport = (0, 0, depth_texture_size[0], depth_texture_size[1])
    ctx.enable(moderngl.DEPTH_TEST)
    ctx.disable(moderngl.CULL_FACE)
    ctx.clear()

    # Compute Light's Views & Projection Matrices.
    light_view_matrix = glm.lookAt(
        light_position,
        glm.vec3(0.0, 0.0, 0.0),
        glm.vec3(0.0, 1.0, 0.0)
    )
    light_proj_matrix = glm.perspective(glm.radians(120.0), 1.0, 1.0, 45.0)

    # Set Uniforms & Render Platform.
    shadow_map_program['model'].write(np.array(model_matrix_platform, dtype='f4').T.tobytes())
    shadow_map_program['view'].write(np.array(light_view_matrix, dtype='f4').T.tobytes())
    shadow_map_program['proj'].write(np.array(light_proj_matrix, dtype='f4').T.tobytes())
    vao_shadow_platform.render()

    # Set Uniforms & Render Teapot.
    shadow_map_program['model'].write(np.array(model_matrix_teapot, dtype='f4').T.tobytes())
    vao_shadow_teapot.render()

    # Re-enable Face Culling After Shadow Mapping.
    ctx.enable(moderngl.CULL_FACE)

    # Pass 2: Render Scene from Camera's Point of View w/ Shadows.
    ctx.screen.use()
    ctx.viewport = (0, 0, width, height)
    ctx.clear(0.1, 0.1, 0.1)
    ctx.enable(moderngl.DEPTH_TEST)

    # Convert View & Project Matrices for Shader Uniform.
    view_bytes = np.array(view_matrix, dtype='f4').T.tobytes()
    proj_bytes = np.array(proj_matrix, dtype='f4').T.tobytes()
    light_view_bytes = np.array(light_view_matrix, dtype='f4').T.tobytes()
    light_proj_bytes = np.array(light_proj_matrix, dtype='f4').T.tobytes()

    depth_texture.use(location=1)

    shadow_map_sampler.use(location=1)

    # Loading Platform w/ Textures.
    grass_texture.use(0)
    program_platform['model'].write(np.array(model_matrix_platform, dtype='f4').T.tobytes())
    program_platform['view'].write(view_bytes)
    program_platform['proj'].write(proj_bytes)
    program_platform['light_position'].value = tuple(light_position)
    program_platform['light_direction'].value = tuple(light_direction)
    program_platform['is_point_light'].value = is_point_light
    program_platform['material_texture'].value = 0
    program_platform['shadow_map'].value = 1  # Texture unit 1
    program_platform['light_view'].write(light_view_bytes)
    program_platform['light_proj'].write(light_proj_bytes)
    vao_platform.render(moderngl.TRIANGLES)

    # Loading Teapot w/ Textures.
    gold_texture.use(0)
    program_teapot['model'].write(np.array(model_matrix_teapot, dtype='f4').T.tobytes())
    program_teapot['view'].write(view_bytes)
    program_teapot['proj'].write(proj_bytes)
    program_teapot['light_position'].value = tuple(light_position)
    program_teapot['light_direction'].value = tuple(light_direction)
    program_teapot['is_point_light'].value = is_point_light
    program_teapot['material_texture'].value = 0
    program_teapot['shadow_map'].value = 1  # Texture unit 1
    program_teapot['light_view'].write(light_view_bytes)
    program_teapot['light_proj'].write(light_proj_bytes)
    program_teapot['metal'].value = True
    program_teapot['eyePosition'].value = tuple(eye_position)
    vao_teapot.render(moderngl.TRIANGLES)

    # Load Light Source.
    model_light_matrix = glm.translate(glm.mat4(1.0), light_position) * light_scale_matrix
    program_light_source['model'].write(np.array(model_light_matrix, dtype='f4').T.tobytes())
    program_light_source['view'].write(view_bytes)
    program_light_source['proj'].write(proj_bytes)
    vao_light_source.render(moderngl.TRIANGLES)

    # Display Depth Texture on Quad.
    H = 200
    W = int(H * aspect_ratio)
    ctx.viewport = (width - W, height - H, W, H)
    quad_program['depth_texture'].value = 1
    depth_texture.use(1)
    vao_quad.render(moderngl.TRIANGLE_STRIP)
    ctx.viewport = (0, 0, width, height)

    pygame.display.flip()

# Clean Up.
pygame.quit()
